In [2]:
# Replicamos la operativa del anterior notebook para mapear e inferir la categoría de distrito a los datos de WI
import numpy as np
import pandas as pd
from collections import Counter
import json
import ijson
import matplotlib.pyplot as plt
import matplotlib.path as mplPath
import seaborn as sns
import folium
from folium.plugins import HeatMap
%pylab
%matplotlib inline

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [9]:
wi = pd.read_csv('wi_with_d.csv', sep=';', encoding='utf-8')

In [10]:
type(wi['district'].iloc[0]), type(wi['district'].iloc[0]), type(wi['district'].iloc[1]), type(wi['district'].iloc[1])

(numpy.int64, numpy.int64, numpy.int64, numpy.int64)

In [11]:
md =  r'Madison_Districts.geojson'
geo_json_data_m = json.load(open(md))

In [12]:
geo_json_data_m

{u'features': [{u'geometry': {u'coordinates': [[[-89.5031697684461,
       43.047355011856396],
      [-89.50318898763713, 43.04622086139778],
      [-89.50032060338748, 43.04624752843616],
      [-89.50032316877974, 43.046130869184715],
      [-89.50033245088, 43.04549307201226],
      [-89.5003430028071, 43.04541778432764],
      [-89.50036504446437, 43.045341842515974],
      [-89.50039588285021, 43.04527168507886],
      [-89.50043763244692, 43.0452024724864],
      [-89.50048901902687, 43.045136841112885],
      [-89.500549478957, 43.045075508717034],
      [-89.50061834669339, 43.045019146320726],
      [-89.50069486950325, 43.044968373699184],
      [-89.50105575354237, 43.044743116707],
      [-89.50114815832026, 43.04467510837649],
      [-89.50123036099352, 43.04460038501439],
      [-89.50130146799154, 43.044519771758594],
      [-89.50136069627392, 43.0444341449759],
      [-89.50140739544094, 43.04434444934762],
      [-89.50144105382465, 43.04425166456029],
      [-89.501

In [13]:
dict_stars_by_district = wi.groupby('district')['stars'].mean()
dict_stars_by_district

district
0     3.660799
1     3.558824
2     3.750000
3     3.430233
4     3.745413
5     3.692982
6     3.976087
7     3.750000
8     3.592000
9     3.490610
10    3.760417
11    3.770408
12    3.578261
13    3.798295
14    3.759740
15    3.822222
16    3.567164
17    3.194561
18    3.857143
19    3.506944
20    3.214286
Name: stars, dtype: float64

In [14]:
centers = wi.groupby('district')[['longitude', 'latitude']].mean()
centers.iloc[5]

longitude   -89.427908
latitude     43.071982
Name: 5, dtype: float64

In [15]:
map_wi = folium.Map(location=[wi['latitude'].mean(),wi['longitude'].mean()], zoom_start=11)

map_wi.choropleth(
    geo_str=geo_json_data_m,
    data=dict_stars_by_district,
    columns=['stars'],
    key_on='properties.ALD_DIST',
    fill_color='RdYlGn',
    #threshold_scale=[3.25, 3.5, 3.75],
    fill_opacity=0.8,
    line_opacity=0.8,
    legend_name='Mean of Stars per District'
)

for i in range(len(centers)):
    if centers.iloc[i].name == 'None':
        pass
    else:
        folium.Marker([centers.iloc[i][1], centers.iloc[i][0]], 
                  popup=str(centers.index[i]), icon=folium.Icon(color='black')).add_to(map_wi)
        
map_wi

In [16]:
dict_bizcount_by_district= wi.groupby('district')['business_id'].count()
dict_bizcount_by_district

district
0     1477
1       34
2      132
3       43
4      436
5       57
6      230
7       12
8      125
9      213
10      48
11      98
12     115
13     176
14      77
15      90
16      67
17     239
18       7
19     216
20       7
Name: business_id, dtype: int64

In [20]:
map_wi = folium.Map(location=[wi['latitude'].mean(),wi['longitude'].mean()], zoom_start=11)

map_wi.choropleth(
    geo_str=geo_json_data_m,
    data=dict_bizcount_by_district,
    columns=['business_id'],
    key_on='properties.ALD_DIST',
    fill_color='YlOrRd',
    threshold_scale=[0, 100, 200, 300, 400],
    fill_opacity=0.8,
    line_opacity=0.8,
    legend_name='Business per District'
)

for i in range(len(centers)):
    if centers.iloc[i].name == 'None':
        pass
    else:
        folium.Marker([centers.iloc[i][1], centers.iloc[i][0]], 
                  popup=str(centers.index[i]), icon=folium.Icon(color='black')).add_to(map_wi)

map_wi

In [21]:
wi.groupby('district')['stars'].mean()[wi.groupby('district').size() > 100].sort_values(ascending=False)

district
6     3.976087
13    3.798295
2     3.750000
4     3.745413
0     3.660799
8     3.592000
12    3.578261
19    3.506944
9     3.490610
17    3.194561
Name: stars, dtype: float64

In [22]:
wi.groupby('district')['stars'].mean()[wi.groupby('district').size() > 100].index

Int64Index([0, 2, 4, 6, 8, 9, 12, 13, 17, 19], dtype='int64', name=u'district')

In [23]:
wi[wi['main_cat'] == 'Restaurants'].groupby('subcat')['business_id'].count()[wi[wi['main_cat'] == 'Restaurants'].groupby('subcat').size() > 15].sort_values(ascending=False)

subcat
Mexican                   83
Pizza                     78
Chinese                   62
American (Traditional)    59
Sandwiches                45
American (New)            44
Italian                   38
Fast Food                 31
Bars                      27
Nightlife                 20
Burgers                   19
Breakfast & Brunch        19
Food                      18
Asian Fusion              16
Name: business_id, dtype: int64

In [24]:
resto = list(wi[wi['main_cat'] == 'Restaurants'].groupby('subcat')['business_id'].count()[wi[wi['main_cat'] == 'Restaurants'].groupby('subcat').size() > 15].sort_values(ascending=False).index)
resto

[u'Mexican',
 u'Pizza',
 u'Chinese',
 u'American (Traditional)',
 u'Sandwiches',
 u'American (New)',
 u'Italian',
 u'Fast Food',
 u'Bars',
 u'Nightlife',
 u'Burgers',
 u'Breakfast & Brunch',
 u'Food',
 u'Asian Fusion']

In [25]:
c = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue',
 'darkpurple', 'pink','gray']

In [26]:
restocol = [[resto[i],c[i]] for i in range(len(resto))]

In [27]:
map_wi = folium.Map(location=[wi['latitude'].mean(),wi['longitude'].mean()], zoom_start=12)

for i in range(len(wi[wi['main_cat'] == 'Restaurants'])):
    if wi[wi['main_cat'] == 'Restaurants']['subcat'].iloc[i] in dict(restocol).keys():
        color = dict(restocol)[wi[wi['main_cat'] == 'Restaurants']['subcat'].iloc[i]]
        folium.Marker([wi[wi['main_cat'] == 'Restaurants']['latitude'].iloc[i], 
                       wi[wi['main_cat'] == 'Restaurants']['longitude'].iloc[i]], 
                      popup=str(wi[wi['main_cat'] == 'Restaurants']['subcat'].iloc[i]), 
                      icon=folium.Icon(color=color)).add_to(map_wi)

folium.GeoJson(geo_json_data_m,
    style_function=lambda feature: {
        'fillColor': '#fff7bc',
        'color': 'black',
        'weight': 2,
        'dashArray': '5, 5'
    }).add_to(map_wi)

map_wi